In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
from keras.datasets import mnist
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

2023-03-20 23:45:31.218378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 23:45:31.664283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 23:45:31.664312: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 23:45:32.861496: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [31]:
class Neural_Network:
    def __init__(self, net):
        self.layers = len(net)
        self.net = net
        self.biases = [np.random.randn(1,k) for k in net[1:]]
        self.weights = [np.random.randn(j,k) for k,j in zip(net[:-1], net[1:])]
        self.c = []
        self.iterate = 0
        
    def get_wb(self):
        return self.weights, self.biases
    
    def SDG(self, data , epochs = 15, mini_batch_size = 500, lr = 3.0, testing_data=False):
        if testing_data:
            return self.evaluation(data)
        batches = [data[i:i+mini_batch_size] for i in range(0,len(data),mini_batch_size)]
        for x in range(epochs):
            np.random.shuffle(data)
            for batch in batches:
                self.mini_batch_update(batch,lr)
            print("epochs: {}/{}".format(x+1, epochs))

    def mini_batch_update(self, batch, lr):
        delta_w = [np.zeros(w.shape) for w in self.weights]
        delta_b = [np.zeros(b.shape) for b in self.biases]
        for i in range(len(batch)):
            input_node = batch[i][0]
            output_node = batch[i][1]
            d_w, d_b = self.backpropagate(input_node, output_node)
            delta_w = [w1+w2 for w1,w2 in zip(delta_w,d_w)]
            delta_b = [b1+b2 for b1,b2 in zip(delta_b, d_b)]

        self.weights = [w-((lr/len(batch))*d_w) for w,d_w in zip(self.weights,delta_w)]
        self.biases = [b-((lr/len(batch))*d_b) for b, d_b in zip(self.biases, delta_b)]    

    def backpropagate(self, x, y):
        # Take in a single training example
        # back propagate
        d_w = [np.zeros(w.shape) for w in self.weights]
        d_b = [np.zeros(b.shape) for b in self.biases]
        z_nodes, a_nodes = self.feedforward(x)
        if (self.iterate %100 ==0):
            self.cost(a_nodes[-1], y)
        output_e = self.delta_cost(a_nodes[-1], y)*self.d_sigmoid(z_nodes[-1])
        d_w[-1] = np.dot(output_e.transpose(), a_nodes[-2])
        d_b[-1] = output_e
        
        for l in range(2,self.layers):
            output_e = np.dot(output_e, self.weights[-l+1])*self.d_sigmoid(z_nodes[-l])
            d_w[-l] = np.dot(output_e.transpose(), a_nodes[-l-1])
            d_b[-l] = output_e
        self.iterate += 1
        return d_w, d_b 
                  
    def feedforward(self, input_node):
        # Take in a training example
        # output the z and a for each layer
        node = np.expand_dims(input_node,0)
        z = []
        a = [node]
        for w,b in zip(self.weights, self.biases):
            node = np.dot(node, w.transpose())+b
            z.append(node)
            node = self.sigmoid(node)
            a.append(node)
        return z, a
    
    def cost(self, output_node, label):
        co = 1/2*(output_node-label)**2
        co = co.mean()
        #print("Cost:", co)
        self.c.append(co)
    
    def get_cost(self):
        return self.c
    
    def delta_cost(self,output_node, label):
        error = (output_node-label)
        return error
    
    def sigmoid(self, z):
        a = 1/(1+np.exp(-z))
        return a
    
    def d_sigmoid(self, z):
        d_a = self.sigmoid(z)*(1-self.sigmoid(z))
        return d_a
    
    def plot_cost(self):
        print("len: {}", len(self.c))
        plt.plot(len(self.c), self.c, 'ro')
        plt.show()
        
    def evaluation(self, data):
        correct = 0
        wrong = 0
        for x,y in zip(data[0], data[1]):
            z,a = self.feedforward(x)
            y_prime = np.argmax(a[-1])
            y = np.argmax(y)

            if y_prime == y:
                correct +=1
            else:
                wrong += 1
        accuracy = correct/len(data)
        print("correct prediction: {} Accuracy: {}".format(correct, accuracy))
            

In [4]:
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

In [26]:
train_x = np.zeros((train_X.shape[0], train_X.shape[1]**2))
train_y = np.zeros((train_Y.shape[0], 10))
test_x = np.zeros((test_X.shape[0], test_X.shape[1]**2)) 
test_y = np.zeros((test_Y.shape[0], 10))

for i in range(len(train_x)):
    x = np.array([train_X[i].ravel()])
    x = np.ma.log2(x)
    x = x.filled(0)
    train_x[i] = x

for i in range(len(train_Y)):
    y = np.zeros(10)
    y[train_Y[i]] = 1
    train_y[i] = y
training_data = [[x,y] for x,y in zip(train_x,train_y)]


for i in range(len(test_x)):
    x = np.array([train_X[i].ravel()])
    x = np.ma.log2(x)
    x = x.filled(0)
    test_x[i] = x

for i in range(len(test_Y)):
    y = np.zeros(10)
    y[test_Y[i]] = 1
    test_y[i] = y
test_data = [[x,y] for x,y in zip(test_x,test_y)]

In [27]:
network = [784,30,10]
epochs = 30
mini_batch_size = 10
learing_rate = 3.0

In [32]:
handwritten_digit_network = Neural_Network(network)

handwritten_digit_network.SDG(training_data, epochs, mini_batch_size, learing_rate)

epochs: 1/30
epochs: 2/30
epochs: 3/30
epochs: 4/30
epochs: 5/30
epochs: 6/30
epochs: 7/30
epochs: 8/30
epochs: 9/30
epochs: 10/30
epochs: 11/30
epochs: 12/30
epochs: 13/30
epochs: 14/30
epochs: 15/30
epochs: 16/30
epochs: 17/30
epochs: 18/30
epochs: 19/30
epochs: 20/30
epochs: 21/30
epochs: 22/30
epochs: 23/30
epochs: 24/30
epochs: 25/30
epochs: 26/30
epochs: 27/30
epochs: 28/30
epochs: 29/30
epochs: 30/30


In [34]:
w,b = handwritten_digit_network.get_wb()

In [ ]:
c = handwritten_digit_network.get_cost()
x = len(c)

for i,y in zip(range(x),c):
    plt.plot(i,y,'ro')
plt.show()

In [33]:
correct = 0
for i in range(len(test_data[:1000])):
    x = test_data[i][0]
    y = test_data[i][1]
    a,z = handwritten_digit_network.feedforward(x)
    x_out = a[-1]
    if np.argmax(x_out) == np.argmax(y):
        correct +=1
print("accuracy: ", correct, len(test_data[:1000]))

accuracy:  94 1000


In [16]:
a = np.array([[1],[2],[3]])
b = np.array([[2,2]])
np.dot(a,b)

array([[2, 2],
       [4, 4],
       [6, 6]])

In [17]:
c = a.transpose()
d = b.transpose()
np.dot(d,c)

array([[2, 4, 6],
       [2, 4, 6]])